<a href="https://colab.research.google.com/github/anjishnu174/Anime-Recommender/blob/main/anime_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [273]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report, accuracy_score


In [274]:
#Loading Data
from google.colab import drive
drive.mount('/content/drive')

# Load your dataset
anime_data = pd.read_csv('/content/drive/MyDrive/anime dataset MAL/AnimeList.csv')
anime_data.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme
0,0,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,When Cowboy Bebop first aired in spring of 199...,Spring 1998,Saturdays at 01:00 (JST),"{'Adaptation': [{'mal_id': 173, 'type': 'manga...",Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","['""Tank!"" by The Seatbelts (eps 1-25)']","['#1: ""The Real Folk Blues"" by The Seatbelts f..."
1,1,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ天国の扉,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,NaN,NaN,NaN,"{'Parent story': [{'mal_id': 1, 'type': 'anime...","Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"Action, Space, Drama, Mystery, Sci-Fi","['""Ask DNA"" by The Seatbelts featuring Raju Ra...","['""Gotta Knock a Little Harder"" by The Seatbel..."
2,2,Trigun,Trigun,トライガン,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,The Japanese release by Victor Entertainment h...,Spring 1998,Thursdays at 01:15 (JST),"{'Adaptation': [{'mal_id': 703, 'type': 'manga...",Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","['""H.T."" by Tsuneo Imahori']","['""Kaze wa Mirai ni Fuku (The Wind Blows to th..."
3,3,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN,WHR,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,NaN,Summer 2002,Tuesdays at Unknown,[],Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Magic, Police, Supernatural, Drama, My...","['""Shell"" by Bana']","['#1: ""Half Pain"" by Bana (eps 01-14,16-25)', ..."
4,4,Beet the Vandel Buster,Beet the Vandel Buster,冒険王ビィト,"Adventure King Beet, Bouken Ou Beet",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,52,Finished Airing,...,NaN,Fall 2004,Unknown,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang...",NaN,NaN,Toei Animation,"Adventure, Fantasy, Shounen, Supernatural","['#1: ""Emotion"" by Sunbrain', '#2: ""Wish Men"" ...","['#1: ""pureness"" by Nana Kitade (eps 1-13)', '..."


In [275]:
anime_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14478 entries, 0 to 14477
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   anime_id        14478 non-null  int64  
 1   title           14478 non-null  object 
 2   title_english   5724 non-null   object 
 3   title_japanese  14443 non-null  object 
 4   title_synonyms  8937 non-null   object 
 5   image_url       14382 non-null  object 
 6   type            14478 non-null  object 
 7   source          14478 non-null  object 
 8   episodes        14478 non-null  int64  
 9   status          14478 non-null  object 
 10  airing          14478 non-null  bool   
 11  aired_string    14478 non-null  object 
 12  aired           14478 non-null  object 
 13  duration        14478 non-null  object 
 14  rating          13934 non-null  object 
 15  score           14478 non-null  float64
 16  scored_by       14478 non-null  int64  
 17  rank            12904 non-null 

In [276]:
anime_data.shape

(14478, 31)

In [277]:
anime_data.describe()

,anime_id,episodes,score,scored_by,rank,popularity,members,favorites
count,14478.000000,14478.000000,14478.000000,1.447800e+04,12904.000000,14478.000000,1.447800e+04,14478.000000
mean,7238.500000,11.308399,6.142482,1.146003e+04,6439.065406,7220.259566,2.296640e+04,311.649606
std,4179.582934,43.443451,1.463981,4.310519e+04,3720.227608,4170.080564,7.498136e+04,2615.554211
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
25%,3619.250000,1.000000,5.550000,4.600000e+01,3216.250000,3611.500000,2.450000e+02,0.000000
50%,7238.500000,1.000000,6.370000,5.010000e+02,6441.500000,7225.500000,1.679500e+03,2.000000
75%,10857.750000,12.000000,7.060000,3.941500e+03,9664.000000,10827.750000,1.037900e+04,23.000000
max,14477.000000,1818.000000,10.000000,1.009477e+06,12919.000000,14487.000000,1.456378e+06,106895.000000


In [278]:
selected_features = ['title', 'title_english', 'title_synonyms',  'source',	'episodes',	'status', 'genre', 'rating', 'score', 'popularity', 'rank' ]
print(selected_features)

['title', 'title_english', 'title_synonyms', 'source', 'episodes', 'status', 'genre', 'rating', 'score', 'popularity', 'rank']


In [279]:
#Check for nulls

anime_data.isnull().sum()

,0
anime_id,0
title,0
title_english,8754
title_japanese,35
title_synonyms,5541
image_url,96
type,0
source,0
episodes,0
status,0


In [280]:
#Data cleaning:
for feature in selected_features:
  anime_data[feature] = anime_data[feature].fillna('')

In [281]:
#Combined features:
combined_features = anime_data['title']+' '+anime_data['title_english']+' '+anime_data['title_synonyms']+' '+anime_data['source'] +' '+anime_data['genre']
print(combined_features)

0        Cowboy Bebop Cowboy Bebop  Original Action, Ad...
1        Cowboy Bebop: Tengoku no Tobira Cowboy Bebop: ...
2        Trigun Trigun  Manga Action, Sci-Fi, Adventure...
3        Witch Hunter Robin Witch Hunter Robin WHR Orig...
4        Beet the Vandel Buster Beet the Vandel Buster ...
                               ...                        
14473          To the Moon   Game Sci-Fi, Adventure, Drama
14474    Senjou no Valkyria Special  Senjou no Valkyria...
14475    Chikyuugai Shounen Shoujo  Extraterrestrial Bo...
14476    Uchuu no Hou: Reimei-hen The Laws of the Unive...
14477    Uma Musume: Pretty Derby (TV) Special   Game S...
Length: 14478, dtype: object


In [282]:
#Vectorizing:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 164953 stored elements and shape (14478, 20545)>
  Coords	Values
  (0, 3298)	0.6477608682901907
  (0, 1594)	0.6931783504862815
  (0, 13273)	0.09473771781918133
  (0, 375)	0.09628446019670331
  (0, 426)	0.10463645746534829
  (0, 3103)	0.077805666605747
  (0, 4329)	0.10866936727222991
  (0, 15640)	0.110169594264242
  (0, 5077)	0.11015260093865703
  (0, 16973)	0.16905982110481152
  (1, 3298)	0.5776939719467953
  (1, 1594)	0.6181987430283489
  (1, 13273)	0.05632676211166175
  (1, 375)	0.05724638517153945
  (1, 4329)	0.06460989076019721
  (1, 15640)	0.0655018578757007
  (1, 5077)	0.06549175441290105
  (1, 16973)	0.10051532320195698
  (1, 17928)	0.1925646573155984
  (1, 12632)	0.052036226972345674
  (1, 18189)	0.19833740560157997
  (1, 18043)	0.05707769245028497
  (1, 11862)	0.08861509345050182
  (1, 9461)	0.22711364948296764
  (1, 20)	0.16339617941114631
  :	:
  (14476, 13273)	0.10240492865764853
  (14476, 15640)	0.11908572108951

In [283]:
#Similarities score:
similarity = cosine_similarity(feature_vectors)
print(similarity)
print(similarity.shape)

[[1.         0.85202236 0.079102   ... 0.08520149 0.06683114 0.        ]
 [0.85202236 1.         0.03420617 ... 0.05065695 0.07078005 0.        ]
 [0.079102   0.03420617 1.         ... 0.06620278 0.0332823  0.        ]
 ...
 [0.08520149 0.05065695 0.06620278 ... 1.         0.09209692 0.        ]
 [0.06683114 0.07078005 0.0332823  ... 0.09209692 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
(14478, 14478)


In [284]:
#User input:
#user_input = input('Enter your anime: ')

In [285]:
#Listing all anime:
# list_of_all_titles = anime_data['title'].tolist()
# print(list_of_all_titles)

In [286]:
#Comparing with similar titles:
# find_close_match = difflib.get_close_matches(user_input, list_of_all_titles)
# print(find_close_match)

In [287]:
#Close match:
# close_match = find_close_match[0]
# print(close_match)

In [288]:
# Indexing close matches:
# index_of_the_anime = anime_data[anime_data.title == close_match]['anime_id'].values[0]
# print(index_of_the_anime)


In [289]:
# List of similar anime (cosine similarity):
# similarity_score = list(enumerate(similarity[index_of_the_anime]))
# print(similarity_score)

In [290]:
#Sorting:
# sorted_similar_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True)
# print(sorted_similar_anime)

In [291]:
'''#For suggestions based on index:
print('Anime suggestions for you: \n')
i=1
for anime in sorted_similar_anime:
   index = anime[0]
   title_from_index = anime_data[anime_data.index==index]['title'].values[0]
   if (i<21):
    print(i,'.', title_from_index)
    i+=1

'''

"#For suggestions based on index:\nprint('Anime suggestions for you: \n')\ni=1\nfor anime in sorted_similar_anime:\n   index = anime[0]\n   title_from_index = anime_data[anime_data.index==index]['title'].values[0]\n   if (i<21):\n    print(i,'.', title_from_index)\n    i+=1\n\n"

Anime Recommender:

In [292]:
user_input = input('Enter your anime: ')

list_of_all_titles = anime_data['title'].tolist()

find_close_match = difflib.get_close_matches(user_input, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_anime = anime_data[anime_data.title == close_match]['anime_id'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_anime]))

sorted_similar_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True)


print('Anime suggestions for you: \n')

i=1
for anime in sorted_similar_anime:
   index = anime[0]
   title_from_index = anime_data[anime_data.index==index]['title'].values[0]
   if (i<21):
    print(i,'.', title_from_index)
    i+=1


Enter your anime: naruto
Anime suggestions for you: 

1 . Naruto
2 . Naruto: Shippuuden
3 . Boruto: Naruto the Movie
4 . Naruto: Shippuuden Movie 1
5 . Naruto: Shippuuden Movie 6 - Road to Ninja
6 . Naruto Movie 2: Dai Gekitotsu! Maboroshi no Chiteiiseki Dattebayo!
7 . Naruto: Shippuuden Movie 5 - Blood Prison
8 . The Last: Naruto the Movie
9 . Naruto: Shippuuden Movie 2 - Kizuna
10 . Naruto x UT
11 . Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi
12 . Naruto: Shippuuden Movie 4 - The Lost Tower
13 . Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!
14 . Naruto: Takigakure no Shitou - Ore ga Eiyuu Dattebayo!
15 . Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono
16 . Naruto: The Cross Roads
17 . Naruto: Honoo no Chuunin Shiken! Naruto vs. Konohamaru!!
18 . Naruto Movie 3: Dai Koufun! Mikazuki Jima no Animaru Panikku Dattebayo!
19 . Naruto Shippuuden: Sunny Side Battle
20 . Boruto: Naruto Next Generations
